## Language Identifier Script
This script is designed to use the psycopg2 Python driver and the langdetect library (must be installed from pip or from conda-forge) to identify which reviews in a selected table are not in English and mark them so they can be analyzed separately.

Currently, the somewhat inelegant solution for deciding which database of reviews is queried is using a find-replace in your editor of choice to name the database. Ideally this will be integrated with an API of some sort, but is currently sufficient for maintaining the dataset.

This script is included for purposes of clarity in demonstrating how the dataset is assembled and managed.

This script has been designed to run in an Anaconda/miniconda virtual environment running Python 3.7.9 with the psycopg2 library (available in all conda channels) and langdetect (available from conda-forge). It will likely work if converted to a script and in other Python versions, but I cannot make any assurances.

It should be noted that the database interface could be more optimized - this script was written with the assumption of lots of processor time to run scripts, to optimize time spent coding in order to get to analysis. There are definitely faster ways to update the database than doing so one row at a time. The script also does not take advantage of multithreading.

In [ ]:
import psycopg2
import langdetect

In [ ]:
try:
    connection = psycopg2.connect() # add your database connection details here
    cursor = connection.cursor()
    cursor_2 = connection.cursor()
    print('Established connection with the database.')
except Exception as e:
    print('exception')
    print(e)

In [ ]:
non_english_reviews = [ ]

cursor.execute('SELECT r_id, review FROM tlj_reviews')

for row in cursor:
    try:
        if langdetect.detect(row[1]) == 'en':
            print(F'Review {row[0]} is English')
        else:
            print(F'Review {row[0]} does not appear to be in English; excluding from dataset.')
            non_english_reviews.append(row[0])
    except:
            print(F'Review {row[0]} does not appear to be in English or may be empty; excluding from dataset.')
            non_english_reviews.append(row[0]) 

In [ ]:
cursor.execute('SELECT r_id, is_english FROM tlj_reviews')
for row in cursor:
    try:
        if row[0] in non_english_reviews:
            cursor_2.execute("UPDATE tlj_reviews SET is_english = FALSE WHERE r_id = (%s);", (row[0], ))
            print(F'Marked {row[0]} as probably not in English.')
            connection.commit()
        else:
            cursor_2.execute("UPDATE tlj_reviews SET is_english = TRUE WHERE r_id = (%s);", (row[0], ))
            connection.commit()
    except Exception as e:
        print(F'Row {row[0]} could not be updated.') 
        print(e)